In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
# How can I plot separate Pandas DataFrames as subplots?
# https://stackoverflow.com/questions/22483588/how-can-i-plot-separate-pandas-dataframes-as-subplots
# Creating multiple subplots using plt.subplots
# https://matplotlib.org/stable/gallery/subplots_axes_and_figures/subplots_demo.html
import re
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import operator as op
matplotlib.style.use('ggplot')
#
regex = re.compile('[^a-zA-Z]')
# Loading the dataset
df = pd.read_csv('../input/kaggle-survey-2021/kaggle_survey_2021_responses.csv', low_memory=False)
descriptions = df.iloc[0,:] # The first row contains the description of the columns, keep it sepearetely
df = df.iloc[1:,:]          # Eliminate the descriptions from the dataframe to ease of use
#Total from dataframe
print('Total people:',df.shape[0])
# drop participants under the following restrictions:
# we excluded respondents that were flagged by our survey system as “Spam” or "Duplicate. 
# I choose these key columns to narrowing the dataframe and considering all of these columns 
# as not merely coincidence but duplication or spam
df=df.drop_duplicates(subset=['Q1', 'Q2', 'Q3','Q4','Q5','Q6','Q8','Q11','Q13','Q15','Q20','Q21'],keep='last')
print('Total people: without SPAM and DUP ... ',df.shape[0])
# We also dropped responses from respondents that spent less than 2 minutes completing the survey, 
df=df[ df.iloc[:, 0].astype('int')>120]      
print('Total people spent more than 2 minutes:',df.shape[0])
# as well as responses from respondents that selected fewer than 15 answer choices in total.
header = df.head().iloc[0:0]   

# clusters of stacked bars with python (Pandas) 
def plot_clustered_stacked(dfall, labels=None, title="/",  H="/", **kwargs):
    """Given a list of dataframes, with identical columns and index, create a clustered stacked bar plot. 
labels is a list of the names of the dataframe, used for the legend
title is a string for the title of the plot
H is the hatch used for identification of the different dataframe"""

    n_df = len(dfall)
    n_col = len(dfall[0].columns) 
    n_ind = len(dfall[0].index)
    axe = plt.subplot(111)

    for df in dfall : # for each data frame
        axe = df.plot(kind="bar",
                      linewidth=0,
                      stacked=True,
                      ax=axe,
                      legend=False,
                      grid=False,
                      **kwargs)  # make bar plots

    h,l = axe.get_legend_handles_labels() # get the handles we want to modify
    for i in range(0, n_df * n_col, n_col): # len(h) = n_col * n_df
        for j, pa in enumerate(h[i:i+n_col]):
            for rect in pa.patches: # for each index
                rect.set_x(rect.get_x() + 1 / float(n_df + 1) * i / float(n_col))
                rect.set_hatch(H * int(i / n_col)) #edited part     
                rect.set_width(1 / float(n_df + 1))

    axe.set_xticks((np.arange(0, 2 * n_ind, 2) + 1 / float(n_df + 1)) / 2.)
    axe.set_xticklabels(df.index, rotation = 75)
    axe.set_title(title)

    # Add invisible data to add another legend
    n=[]        
    for i in range(n_df):
        n.append(axe.bar(0, 0, color="lightgray", hatch=H * i))

    l1 = axe.legend(h[:n_col], l[:n_col], loc=[1.01, 0.5])
    if labels is not None:
        l2 = plt.legend(n, labels, loc=[1.01, 0.1]) 
    axe.add_artist(l1)
    return axe
#
def _clustered_stacked_by_country_gender(country, gender,title="/"):
    Ages =[]
    Genders=[]
    Countries=[]
    Formal_Grade=[]
    Current_Rule_Title=[]
    How_Many_YeasCode=[]
    #
    What_Programming_Python=[]
    What_Programming_R=[]
    What_Programming_SQL=[]
    What_Programming_C=[]
    What_Programming_CPP=[]
    What_Programming_Java=[]
    What_Programming_JS=[]
    What_Programming_Julia=[]
    What_Programming_Swift=[]
    What_Programming_Bash=[]
    What_Programming_MATLAB=[]
    What_Programming_None=[]
    What_Programming_Other=[]
    #
    Programming_Lang_Recomend=[]
    # Q9_Part_1 - Q9_Part_12 - Q9_OTHER
    IDE_Jupyter=[]
    IDE_RStudio=[]
    IDE_VidualStudio=[]
    IDE_VSCode=[]
    IDE_PyCharm=[]
    IDE_Spyder=[]
    IDE_Npp=[]
    IDE_SublimeText=[]
    IDE_Vim_Emacs=[]
    IDE_MATLAB=[]
    IDE_None=[]
    IDE_Other=[]    
    # Q10_Part_1 - Q10_Part_13 - Q10_OTHER
    Notebook_Kaggle=[]
    Notebook_Colab=[]
    Notebook_Azure=[]
    Notebook_Paperspace=[]
    Notebook_Binder=[]
    Notebook_CodeOcean=[]
    Notebook_IBM_WhatsonStudio=[]
    Notebook_SageMakerAMS=[]
    Notebook_EMR_AWS=[]
    Notebook_GCAI_Google=[]
    Notebook_GCD_Google=[]
    Notebook_Databricks=[]
    Notebook_None=[]
    Notebook_OTHER=[]
    # Q10_Part_1 - Q10_Part_13 - Q10_OTHER
    Type_of_Comp_Platfrm=[]
    # Q12_Part_1 - Q12_Part_3 - Q12_OTHER    
    Type_Special_Hrdwr_GPUs=[]
    Type_Special_Hrdwr_TPUs=[]
    Type_Special_Hrdwr_None=[]
    Type_Special_Hrdwr_OTHER=[]
    #
    #
    TPU_How_Many_Times=[]
    #What_Data_Visualztion_Libs=[]
    #
    Countries.append(country)
    # Iterate over each row 
    # Hierarchy -> Country - Formal Grade - Age - Gender
    # this way of organization will bring more similarity 
    # for grouping data
    lstAgesRange = list(set(df['Q1'] ))
    lstFormalGrade = list(set(df['Q4'] ))
    
    countAgeRanges = len(lstAgesRange)
    countFormalGrade = len(lstFormalGrade)
    #Filtered by gender and country
    print('Ages range',lstAgesRange)
    dfaux = df.loc[df['Q2'] == gender]
    dfaux2 = dfaux.loc[dfaux['Q3'] == country]
    for index, rows in dfaux2.iterrows(): 
        # Create list for the current row 
        #print(index,') - Age: ',rows.Q1,' Gender: ',rows.Q2,' Country: ',rows.Q3,' Formal grade: ',rows.Q4,' Job Title: ',rows.Q5,' Years of expert: ',rows.Q6,' Coding lang: ',rows.Q7_OTHER)
        Ages.append(rows.Q1)
        Genders.append(rows.Q2)
        Formal_Grade.append(rows.Q4)
        Current_Rule_Title.append(rows.Q5)
        How_Many_YeasCode.append(rows.Q6)
        # Q7_Part_1 - Q7_Part_12 - Q7_OTHER
        What_Programming_Python.append(rows.Q7_Part_1)
        What_Programming_R.append(rows.Q7_Part_2)
        What_Programming_SQL.append(rows.Q7_Part_3)
        What_Programming_C.append(rows.Q7_Part_4)
        What_Programming_CPP.append(rows.Q7_Part_5)
        What_Programming_Java.append(rows.Q7_Part_6)
        What_Programming_JS.append(rows.Q7_Part_7)
        What_Programming_Julia.append(rows.Q7_Part_8)
        What_Programming_Swift.append(rows.Q7_Part_9)
        What_Programming_Bash.append(rows.Q7_Part_10)
        What_Programming_MATLAB.append(rows.Q7_Part_11)
        What_Programming_None.append(rows.Q7_Part_12)
        What_Programming_Other.append(rows.Q7_OTHER)
        # Q8 - programming language recomendation
        Programming_Lang_Recomend.append(rows.Q8)
        # Q9_Part_1 - Q9_Part_11 - Q9_OTHER
        IDE_Jupyter.append(rows.Q9_Part_1)
        IDE_RStudio.append(rows.Q9_Part_2)
        IDE_VidualStudio.append(rows.Q9_Part_3)
        IDE_VSCode.append(rows.Q9_Part_4)
        IDE_PyCharm.append(rows.Q9_Part_5)
        IDE_Spyder.append(rows.Q9_Part_6)
        IDE_Npp.append(rows.Q9_Part_7)
        IDE_SublimeText.append(rows.Q9_Part_8)
        IDE_Vim_Emacs.append(rows.Q9_Part_9)
        IDE_MATLAB.append(rows.Q9_Part_10)
        IDE_None.append(rows.Q9_Part_11)
        IDE_Other.append(rows.Q9_OTHER)         
        # Q10_Part_1 - Q10_Part_13 - Q10_OTHER
        Notebook_Kaggle.append(rows.Q10_Part_1)
        Notebook_Colab.append(rows.Q10_Part_2)
        Notebook_Azure.append(rows.Q10_Part_3)
        Notebook_Paperspace.append(rows.Q10_Part_4)
        Notebook_Binder.append(rows.Q10_Part_5)
        Notebook_CodeOcean.append(rows.Q10_Part_6)
        Notebook_IBM_WhatsonStudio.append(rows.Q10_Part_7)
        Notebook_SageMakerAMS.append(rows.Q10_Part_8)
        Notebook_EMR_AWS.append(rows.Q10_Part_9)
        Notebook_GCAI_Google.append(rows.Q10_Part_10)
        Notebook_GCD_Google.append(rows.Q10_Part_11)
        Notebook_Databricks.append(rows.Q10_Part_12)
        Notebook_None.append(rows.Q10_Part_13)
        Notebook_OTHER.append(rows.Q10_OTHER)
        #
        Type_of_Comp_Platfrm.append(rows.Q11)
        # Q12_Part_1 - Q12_Part_3 - Q12_OTHER
        Type_Special_Hrdwr_GPUs.append(rows.Q12_Part_1)
        Type_Special_Hrdwr_TPUs.append(rows.Q12_Part_2)
        Type_Special_Hrdwr_None.append(rows.Q12_Part_3)
        Type_Special_Hrdwr_OTHER.append(rows.Q12_OTHER)                            
        #
        TPU_How_Many_Times.append(rows.Q13)
        #What_Data_Visualztion_Libs.append(rows.Q14)
    #   
    AgesOccurs=[]
    #print(Genders)
    #print(Countries)
    Formal_GradeOccurs=[]
    #
    lstAgesRange.sort()
    #For item in Ages
    for item in lstAgesRange:
        AgesOccurs.append(op.countOf(Ages,item))
    #
    AgesOccurs.sort()
    #For item in Grades
    for item in lstFormalGrade:
        Formal_GradeOccurs.append(op.countOf(Formal_Grade,item))
    #
    #_lstFormalGrade = [regex.sub('', FormalGrade) for FormalGrade in lstFormalGrade]
    
    _lstFormalGrade = ['Master Degree', 'No Formal', 'Preffer Not Answer', 'Prof Doctorate', 'Some College', 'Doctoral Degree', 'Bachelors Degree']
    ##  
    #if (len(lstAgesRange) > len(_lstFormalGrade)):
    #    lower = len(_lstFormalGrade)-1
    #    upper = len(lstAgesRange)-1
    #    while lower<upper:
    #        _lstFormalGrade.append('')
    #        Formal_GradeOccurs.append('0')
    #        lower+=1
    #        
    #elif (len(_lstFormalGrade) >len(lstAgesRange)):
    #    lower = len(lstAgesRange)-1
    #    upper = len(_lstFormalGrade)-1
    #    while lower<upper:
    #        lstAgesRange.append('')
    #        AgesOccurs.append(0)
    #        lower+=1
    ##
    # don't use the series but it's metrics

    print('Ages ranges: ',len(lstAgesRange),' ',lstAgesRange)
    print('Ages Occurences: ',len(AgesOccurs),' ',AgesOccurs)
    print('Education ranges: ',len(_lstFormalGrade),' ',_lstFormalGrade)
    print('Education Occurences: ',len(Formal_GradeOccurs),' ',Formal_GradeOccurs)
    
    print('Maximum elements: ',max(len(lstAgesRange), len(_lstFormalGrade)),' - ', (' Ages ' if len(lstAgesRange) == max(len(lstAgesRange), len(_lstFormalGrade)) else ' Formal education'))
    # create fake dataframes
    df1 = pd.DataFrame(np.random.rand(len(lstAgesRange), len(AgesOccurs)),
                       AgesOccurs,
                       lstAgesRange)
    #
    df2 = pd.DataFrame(np.random.rand(len(_lstFormalGrade), len(Formal_GradeOccurs)),
                       Formal_GradeOccurs,
                       _lstFormalGrade)
    #
    df3 = pd.DataFrame(np.random.rand(len(_lstFormalGrade), len(lstAgesRange)),
                       _lstFormalGrade,
                       lstAgesRange)

    # Then, just call :
    print('Plotting for: ',country,' ',gender)
    #plot_clustered_stacked([df1, df2],["Ages", "Formal Grade"])
    #plot_clustered_stacked([df1],["Ages"])
    plot_clustered_stacked([df3],["Age vs Formal Grade"],title)
    #
 #
print('1')
_clustered_stacked_by_country_gender('India', 'Woman',"India women statistics: Age & Formal Education")
print('2')
_clustered_stacked_by_country_gender('India', 'Man',"India men statistics: Age & Formal Education")
print('3')
_clustered_stacked_by_country_gender('Brazil', 'Woman',"Brazil women statistics: Age & Formal Education")
print('4')
_clustered_stacked_by_country_gender('Brazil', 'Man',"Brazil men statistics: Age & Formal Education")

# People participated to the survey
number_of_people = df.shape[0]
number_of_questions_answered = df.shape[1]
print('Number of people participated to the survey:', df.shape[0])
print('Number of questions answered:', df.shape[1])

# Turkish people participated to the survey
my_country = 'Brazil'
GCP_familiar = ' Google Cloud Platform (GCP) '
AZURE_Familiar = ' Microsoft Azure '
my_folks = 'Brazilian'
df_Brazil = df[df.Q3 == my_country]
df_GCloud_Familiar = df.loc[df['Q28'] == GCP_familiar]
df_AZURE_Familiar = df.loc[df['Q28'] == AZURE_Familiar]
df_NotCloud_Familiar = df.loc[(df['Q28'].empty) or (pd.isna(df['Q28']))]


number_of_br_people = df_Brazil.shape[0]
number_of_cloud_familiar_people = df_GCloud_Familiar.shape[0]
number_of_azure_familiar_people = df_AZURE_Familiar.shape[0]
number_of_cloud_non_familiar_people = df_NotCloud_Familiar.shape[0]
print(f'Number of {my_folks} people participated to the survey:', number_of_br_people)
print(f'Number of people participated to the survey who are familiar with Google Cloud Platform:', number_of_cloud_familiar_people)
print(f'Number of people participated to the survey who are familiar with AZURE Cloud Platform:', number_of_azure_familiar_people)
print(f'Number of people participated to the survey who are not familiar with any Cloud Platform:', number_of_cloud_non_familiar_people)

question = descriptions.iloc[3]    # Question
country = df.Q3
x = country.value_counts().values
y = country.value_counts().index

# Only explode the slice of my country.
explode = (y == my_country) * 0.15

# Only show labels above 1 percent, to avoid overlapping
percent = 100.*x/x.sum()
labels = ['{0} - {1:1.2f}%'.format(i,j) if j > 1.0 else '' for i, j in zip(y, percent)]

plt.subplots(figsize=[24,14])
patches, texts = plt.pie(x, labels=labels, explode=explode, startangle=90, labeldistance=1.05, textprops={'fontsize': 14})

# More labels can be shown in the legend without any overlapping
legend_labels = ['{0} - {1:1.2f}%'.format(i,j) for i, j in zip(y, percent) if j > 0.5]
plt.legend(patches, legend_labels, loc="upper left")

# Set aspect ratio to be equal so that pie is drawn as a circle.
plt.axis('equal')
plt.tight_layout()

percent_of_my_country = 100.*x[y.get_loc(my_country)]/x.sum()
plt.title('{0}\n{1:1.2f}% of Kagglers are from {2}'.format(question, percent_of_my_country, my_country), fontsize=24);
plt.show()
# Countries which are not matching with geopandas map
countries = df[df.columns[3]].copy()
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

countries_not_matching = []
for name in y:
    if sum(world['name'] == name) == 0:
        countries_not_matching.insert(len(countries_not_matching), name)
print(countries_not_matching)
# Replace and remove countries
countries.replace({'United Kingdom of Great Britain and Northern Ireland':'United Kingdom',
                   'Iran, Islamic Republic of...':'Iran',
                   'Czech Republic':'Czechia',
                   'Viet Nam':'Vietnam'}, inplace=True)
x = countries.value_counts().values
y = countries.value_counts().index

countries_not_matching = ['Other', 'Singapore', 'Hong Kong (S.A.R.)', 'I do not wish to disclose my location']
for country in countries_not_matching:
    index = y.get_loc(country)
    y = y.delete(index)
    x = np.delete(x, index)
# Reading the geopandas data 
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Next we need to create a dataframe with lis_countries and lis_pop
column = 'kaggle_population'
country_data = pd.DataFrame(y, columns=['Country'])
country_data[column] = x

# Next, we are going to visualize this...
world = world.set_index('name').join(country_data.set_index('Country')).reset_index()
fig, ax = plt.subplots(figsize=(24,16))

world.plot(column=column, ax=ax, legend=True, legend_kwds={'label': "Kaggle population by country", 'orientation': "horizontal", 'shrink': 0.3})
ax.set_axis_off();
# AGE
my_age = 53

question = descriptions.iloc[1]    # Question
age = df.Q1
x = age.value_counts().values
y = age.value_counts().index

# Only explode the slice of my age
ages = [age.replace('+', '').split('-') for age in y]
ages[-1] = [ages[-1][0], 'inf'] # 70+ age doesn't have a max value
explode = [0.1 if float(age[0]) <= my_age and float(age[1]) >= my_age else 0 for age in ages]

# Only show labels above 1 percent, to avoid overlapping
percent = 100.*x/x.sum()
labels = ['{0} - {1:1.2f} %'.format(i,j) if j > 1.0 else '' for i, j in zip(y, percent)]

plt.subplots(figsize=[24,14])
patches, texts = plt.pie(x, labels=labels, explode=explode, labeldistance=1.03, textprops={'fontsize': 14})

# Add a legend
legend_labels = ['{0} - {1:1.2f} %'.format(i,j) for i, j in zip(y, percent) if j > 0.5]
plt.legend(patches, legend_labels, loc="upper left", prop={'size': 16})

# Set aspect ratio to be equal so that pie is drawn as a circle.
plt.axis('equal')
plt.tight_layout()

my_age_range = [float(age[0]) <= my_age and float(age[1]) >= my_age for age in ages].index(True)
percent_of_my_age = 100.*x[my_age_range]/x.sum()
plt.title('{0}\n{1:1.2f} % of Kagglers are in your age.'.format(question, percent_of_my_age), fontsize=24);
plt.show()

